In [10]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values

-- adding necessary JARs and including database
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

-- Create external table 
Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)
-- Your code here
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts';

Overwriting task1_create_external_table.hql


In [11]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

Overwriting task1_create_managed_table.hql


In [12]:
%%writefile task1_insert_table.hql

SET hive.exec.dynamic.partition=true;
SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=500;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.error.on.empty.partition=true;

-- adding necessary JARs and including database
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

-- Insert data to the managed table

USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, year, month
WHERE year IS NOT NULL and month is not NULL;

Overwriting task1_insert_table.hql


In [13]:
%%writefile task1_result.hql
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;

select concat_ws("\t", year, concat_ws("-", year, month), row_cnt)
from(
SELECT year, month,STRING(count(*)) as row_cnt, ROW_NUMBER() OVER (ORDER BY year, month) as row_nbr
FROM posts_sample
GROUP BY year, month
) as temp
where row_nbr = 3

Overwriting task1_result.hql


In [14]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

In [15]:
!cat task1.hql

-- Create external table posts_sample_external with suitable values

-- adding necessary JARs and including database
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

-- Create external table 
Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)
-- Your code here
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts';
-- create managed table
-- Check that this table contains info about year and month

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

SET hive.exec.dynamic.partition=true;
SET hive.ex

In [16]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [17]:
%%bash
hive -f task1.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-2.3.6-bin/lib/hive-common-2.3.6.jar!/hive-log4j2.properties Async: true
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.712 seconds
OK
Time taken: 0.225 seconds
OK
Time taken: 0.144 seconds
OK
Time taken: 0.027 seconds
OK
Time taken: 0.129 seconds
OK
Time taken: 0.13 seconds
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 0.023 seconds
Query ID = jovyan_20201216001935_e030d388-8f38